# GA Practice - Q2

We combined two datasets in single CSV file. Use the below script to seperate the data.

Use data_dime for dimensionality reduction and data_recom for recommendation system.

import pandas as pd

data=pd.read_csv('dataset3.csv')

data_dime=data.iloc[0:5891,0:30]

data_recom=data.iloc[:,30:34]

## SECTION A - (5 marks)
### 1. Data Preprocessing 

    a. Read the dataset and perform required cleaning and preprocessing prior to model building. (1 MARK)
    
    b. Calculate five-point summary for numerical variables. Summarize observations for categorical variables – no. of categories, % observations in each category. (1 MARK)
    
    c. Perform univariate and bivariate analysis (2 MARKS)
    
    d. Scale / Transform/ clean the data so that it is suitable for model building. Drop "SalePrice" before using clustering methods, as this is the target attribute. (1 MARK)

## SECTION B (20 marks)
### 2. Answer the following questions 

    a. Use inp_data_dime. Apply PCA and compute all the possible principle components (PCs). How many PCs are required to reproduce the 95% characteristics of original data. Plot it with appropriate diagram. Also print the top 5 eigen vectors (5 marks)

    b. Create a random matrix (M) of size 20 x 8 and compute singular values, left singular matrix and right singular matrix using Singular Value Decomposition. Try to reproduce the M back using singular values and vectors. (5 Marks)

    c. Apply SVD on inp_data_dime and compare the SVD transform data with PCA transformed data. Also compare the top 5 singular vectors with eigen vector. How many Singular vectors are required to reproduce the 95% charecteristics of original data. (5 Marks)

    d. Clustering: Use PCA dimensions to cluster the data. Apply K-means and Agglomerative clustering. (5 Marks)

## SECTION C (15 marks)
### 3. Recommendation Systems 
    a. Build the popularity-based recommendation system and suggest top 5 items. (5 Marks)
    
    b. Build collaborative recommendation engine to recommend a top product/item to the specific user. Measure the model quality in terms of RMSE. (10 Marks)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage , dendrogram, fcluster,cophenet
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from surprise import KNNWithMeans,SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split,cross_validate
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'surprise'

## SECTION A 
### 1. Data Preprocessing 

### a. Read the dataset and perform required cleaning and preprocessing prior to model building.

In [ ]:
# Read the dataset
data = pd.read_csv("C:\Datasets\dataset3.csv")

In [ ]:
data.shape

In [ ]:
# Separate the datasets
data_dime = data.iloc[0:5891, 0:30]
data_recom = data.iloc[:, 30:34]

In [ ]:
print("Data for Dimensionality Reduction:")
data_dime.head()

In [ ]:
data_dime.info()

In [ ]:
print("\nData for Recommendation System:")
data_recom.head()

In [ ]:
data_recom.info()

### b. Calculate five-point summary for numerical variables. Summarize observations for categorical variables – no. of categories, % observations in each category.

In [ ]:
# Five-point summary for numerical variables
print("Five-point summary for numerical variables:")
data_dime.describe()

In [ ]:
# Summarize observations for categorical variables
categorical_summary = data_dime.select_dtypes(include=['object']).apply(lambda x: x.value_counts(normalize=True) * 100)
print("\nSummary for categorical variables:")
categorical_summary

### c. Perform univariate and bivariate analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Univariate analysis
for column in data_dime.select_dtypes(include=['number']).columns:
    sns.histplot(data_dime[column], kde=True)
    plt.title(f'Univariate Analysis of {column}')
    plt.show()

In [ ]:
# Bivariate analysis
sns.pairplot(data_dime.select_dtypes(include=['number']))
plt.title('Bivariate Analysis')
plt.show()

### d. Scale / Transform/ clean the data so that it is suitable for model building. Drop “SalePrice” before using clustering methods, as this is the target attribute.

In [ ]:
# Drop the target attribute "SalePrice"
df = data_dime.drop(columns=['SalePrice'])
df_num = data_dime.select_dtypes(include=np.number)
df_cat = data_dime.select_dtypes(include=[object])

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in df_cat.columns:
    df_cat[i] = le.fit_transform(df_cat[i])

In [ ]:
df1 = pd.concat([df_num,df_cat],axis=1)

In [ ]:
# Scaling the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df1)
scaled_data

## SECTION B (20 marks)

### a. Apply PCA and compute all the possible principle components (PCs). How many PCs are required to reproduce the 95% characteristics of original data. Plot it with appropriate diagram. Also print the top 5 eigen vectors.

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=0.95)  # Keep 95% of variance
pca_scaled_data = pca.fit_transform(scaled_data)

# Number of components to capture 95% variance
num_components = pca.n_components_
print(f"Number of components to capture 95% variance: {num_components}")

In [ ]:
# Compute explained variance
explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot explained variance
plt.figure(figsize=(10, 6))
plt.plot(explained_variance,marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Explained Variance')
plt.title('Explained Variance vs Number of Principal Components')
plt.grid(True)
plt.show()

In [ ]:
# Print the top 5 eigenvalues
top_eigenvalues = pca.explained_variance_[:5]
print("Top 5 Eigenvalues:")
print(top_eigenvalues)

In [ ]:
# Print the top 5 eigenvectors
top_eigenvectors = pca.components_[:5]
print("\nTop 5 Eigenvectors:")
print(top_eigenvectors)

### b. Create a random matrix (M) of size 20 x 8 and compute singular values, left singular matrix and right singular matrix using Singular Value Decomposition. Try to reproduce the M back using singular values and vectors.

In [ ]:
import numpy as np

# Create a random matrix
M = np.random.rand(20, 8)

# Compute SVD
U, S, VT = np.linalg.svd(M)

# Reconstruct M using SVD components
# S should be expanded to a diagonal matrix of shape (8, 8)
S_diag = np.diag(S)

# Reconstruct the original matrix M
M_reconstructed = U[:, :8] @ S_diag @ VT

print("Original Matrix M:")
print(M)

print("\nReconstructed Matrix M:")
print(M_reconstructed)

# Check if reconstruction is close to the original
reconstruction_error = np.linalg.norm(M - M_reconstructed)
print(f"Reconstruction error: {reconstruction_error}")


### c. Apply SVD on inp_data_dime and compare the SVD transform data with PCA transformed data. Also compare the top 5 singular vectors with eigen vector. How many Singular vectors are required to reproduce the 95% charecteristics of original data.

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Apply SVD
svd = TruncatedSVD(n_components=num_components)
data_dime_svd = svd.fit_transform(scaled_data)

In [ ]:
# Compare top 5 singular vectors with eigen vectors
top_5_singular_vectors = svd.components_[:5]
print("Top 5 singular vectors:")
print(top_5_singular_vectors)

In [ ]:
# Number of components to explain 95% variance in SVD
svd_explained_variance = svd.explained_variance_ratio_
num_svd_components = next(i for i, total_var in enumerate(svd_explained_variance.cumsum()) if total_var >= 0.95) + 1
print(f"Number of SVD components to explain 95% variance = {num_svd_components}")

### d. Clustering: Use PCA dimensions to cluster the data. Apply K-means and Agglomerative clustering.

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering

# Use PCA dimensions for clustering
pca_dims = pca_scaled_data[:, :num_components]

# K-means clustering
kmeans = KMeans(n_clusters=5, n_init=10)
kmeans_labels = kmeans.fit_predict(pca_dims)
print("K-means clustering labels:")
print(kmeans_labels)

In [ ]:
# Agglomerative clustering
agglo = AgglomerativeClustering(n_clusters=5)
agglo_labels = agglo.fit_predict(pca_dims)
print("\nAgglomerative clustering labels:")
print(agglo_labels)

## SECTION C 
### 3. Recommendation Systems

In [ ]:
data_recom.info()

### a. Build the popularity-based recommendation system and suggest top 5 items.

In [ ]:
import pandas as pd

data_recom.columns = ['UserID', 'ItemID', 'Rating','Timestamp']

# Calculate the average rating and the count of ratings for each item
item_popularity = data_recom.groupby('ItemID').agg({'Rating': ['mean', 'count']})
item_popularity.columns = ['Average_rating', 'Rating_count']

# Sort items by rating count and mean rating
top_items = item_popularity.sort_values(by=['Rating_count', 'Average_rating'], ascending=False).head(5)

print("Top 5 popular items:")
top_items

### b. Build collaborative recommendation engine to recommend a top product/item to the specific user. Measure the model quality in terms of RMSE.

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Prepare data for collaborative filtering
reader = Reader(rating_scale=(data_recom['Rating'].min(), data_recom['Rating'].max()))
data_cf = Dataset.load_from_df(data_recom[['UserID', 'ItemID', 'Rating']], reader)

In [ ]:
trainset, testset = train_test_split(data_cf, test_size=0.2)

### (i) Build a user-based collaborative filtering model

In [ ]:
from surprise import Dataset, Reader, KNNBasic

sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

In [ ]:
# Evaluate RMSE

rmse = accuracy.rmse(predictions)

In [ ]:
# Function to recommend top N items for a specific user
def get_top_n_recommendations(algo, user_id, n=1):
    # Get a list of all item ids
    all_items = data_recom['ItemID'].unique()
    # Get a list of items already rated by the user
    rated_items = data_recom[data_recom['UserID'] == user_id]['ItemID']
    # Get a list of items not yet rated by the user
    items_to_recommend = [item for item in all_items if item not in rated_items]
    
    # Predict the rating for each item not yet rated
    predictions = [algo.predict(user_id, item_id) for item_id in items_to_recommend]
    
    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # Return the top N items
    top_n_items = predictions[:n]
    return top_n_items

In [ ]:
# Example usage
user_id = data_recom['UserID'].iloc[10]  # Replace with a specific user id
top_recommendations = get_top_n_recommendations(model, user_id, n=2)

print(f"Top recommendation for user {user_id}:")
for recommendation in top_recommendations:
    print(f"Item {recommendation.iid} with predicted rating {recommendation.est}")

### (ii) Using SVD for collaborative filtering

In [ ]:
model = SVD(n_factors=50, n_epochs=200)
model.fit(trainset)

In [ ]:
predictions = model.test(testset)

In [ ]:
rmse = accuracy.rmse(predictions)

In [ ]:
# Function to recommend top N items for a specific user
def get_top_n_recommendations(algo, user_id, n=1):
    # Get a list of all item ids
    all_items = data_recom['ItemID'].unique()
    # Get a list of items already rated by the user
    rated_items = data_recom[data_recom['UserID'] == user_id]['ItemID']
    # Get a list of items not yet rated by the user
    items_to_recommend = [item for item in all_items if item not in rated_items]
    
    # Predict the rating for each item not yet rated
    predictions = [algo.predict(user_id, item_id) for item_id in items_to_recommend]
    
    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # Return the top N items
    top_n_items = predictions[:n]
    return top_n_items

In [2]:
# Example usage
user_id = data_recom['UserID'].iloc[2]  # Replace with a specific user id
top_recommendations = get_top_n_recommendations(model, user_id, n=5)

print(f"Top recommendation for user {user_id}:")
for recommendation in top_recommendations:
    print(f"Item {recommendation.iid} with predicted rating {recommendation.est}")

NameError: name 'data_recom' is not defined